# Use Case: Electricity Consumption of a single building

**Dataset**: Half-hourly electricity consumption data from January to April 2024 enriched with weather conditions, historical consumptions and anomaly labels.  
[48 intervals in a day]

---

## Research Questions

1. How is the electricity consumption from winter to spring?  
2. Can we deduce if the building is a residential or a non-residential building?  
3. Does the change of electricity consumption is related to the weather (temperature, wind and/or humidity)?

## Why Data Aggregation Matters

In this section, we'll explore how aggregating high-frequency consumption data (half-hourly) to different time scales (hourly, daily, weekly) to reduce noise in the data and answer  questions at the appropriate time scale

In [11]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.dates as mdates

# 1. Import data and data preparation

In [3]:
df = pd.read_csv(r"datasets/smart_meter_data.csv")
df

,Timestamp,Electricity_Consumed,Temperature,Humidity,Wind_Speed,Avg_Past_Consumption,Anomaly_Label
0,2024-01-01 00:00:00,0.457786,0.469524,0.396368,0.445441,0.692057,Normal
1,2024-01-01 00:30:00,0.351956,0.465545,0.451184,0.458729,0.539874,Normal
2,2024-01-01 01:00:00,0.482948,0.285415,0.408289,0.470360,0.614724,Normal
3,2024-01-01 01:30:00,0.628838,0.482095,0.512308,0.576241,0.757044,Normal
4,2024-01-01 02:00:00,0.335974,0.624741,0.672021,0.373004,0.673981,Normal
...,...,...,...,...,...,...,...
4995,2024-04-14 01:30:00,0.366839,0.701004,0.362397,0.509174,0.490516,Normal
4996,2024-04-14 02:00:00,0.493568,0.258212,0.677895,0.627889,0.535212,Normal
4997,2024-04-14 02:30:00,0.893818,0.431739,0.688926,0.508038,0.681099,Abnormal
4998,2024-04-14 03:00:00,0.509673,0.592927,0.366151,0.668218,0.710599,Normal


In [12]:
df.Temperature.min(), df.Temperature.max()

(0.0, 1.0)

In [13]:
# Convert 'Timestamp' column to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.dtypes


Timestamp               datetime64[ns]
Electricity_Consumed           float64
Temperature                    float64
Humidity                       float64
Wind_Speed                     float64
Avg_Past_Consumption           float64
Anomaly_Label                   object
Year                             int32
Month                            int32
Day                              int32
Day_name                        object
week_day                         int32
dtype: object

## Extract year, month, day and day name 
- Extract: year, month, day (1:30/28/31), day name, day_of_year (1:105) by using dt.year or dt.strftime()  
- Extract: week, day_2, hour by using dt.to_period().apply(lambda r: r.start_time)  

NB: .apply(lambda r: r.start_time) is to have datetime format and not period  (otehrwise mtpl or seaborn cannot plot)

In [130]:
#Methdology 2: dt accessor 

df["Year"]=df["Timestamp"].dt.year
df["Month"]=df["Timestamp"].dt.month
df["Month_name"]=df["Timestamp"].dt.strftime("%B")
df["Day"]=df["Timestamp"].dt.day
df["Day_name"]=df["Timestamp"].dt.day_name()
df["Day_of_year"]=df["Timestamp"].dt.day_of_year 
df['Day_Type'] = df['Timestamp'].dt.dayofweek.apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')
df['Week'] = df['Timestamp'].dt.to_period('W').apply(lambda r: r.start_time)
df["Day_2"]=df["Timestamp"].dt.to_period("D").apply(lambda r: r.start_time)
df["Hour"]=df["Timestamp"].dt.to_period("H").apply(lambda r: r.start_time)
df

/var/folders/77/thbhtfx5753f1j0bt0__myth0000gn/T/ipykernel_3866/3314045387.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["Hour"]=df["Timestamp"].dt.to_period("H").apply(lambda r: r.start_time)


,Timestamp,Electricity_Consumed,Temperature,Humidity,Wind_Speed,Avg_Past_Consumption,Anomaly_Label,Year,Month,Day,Day_name,week_day,Hour,Week,Day_2,Day_of_year,Day_Type,Month_name
0,2024-01-01 00:00:00,0.457786,0.469524,0.396368,0.445441,0.692057,Normal,2024,1,1,Monday,0,2024-01-01 00:00:00,2024-01-01,2024-01-01,1,Weekday,January
1,2024-01-01 00:30:00,0.351956,0.465545,0.451184,0.458729,0.539874,Normal,2024,1,1,Monday,0,2024-01-01 00:00:00,2024-01-01,2024-01-01,1,Weekday,January
2,2024-01-01 01:00:00,0.482948,0.285415,0.408289,0.470360,0.614724,Normal,2024,1,1,Monday,0,2024-01-01 01:00:00,2024-01-01,2024-01-01,1,Weekday,January
3,2024-01-01 01:30:00,0.628838,0.482095,0.512308,0.576241,0.757044,Normal,2024,1,1,Monday,0,2024-01-01 01:00:00,2024-01-01,2024-01-01,1,Weekday,January
4,2024-01-01 02:00:00,0.335974,0.624741,0.672021,0.373004,0.673981,Normal,2024,1,1,Monday,0,2024-01-01 02:00:00,2024-01-01,2024-01-01,1,Weekday,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-04-14 01:30:00,0.366839,0.701004,0.362397,0.509174,0.490516,Normal,2024,4,14,Sunday,6,2024-04-14 01:00:00,2024-04-08,2024-04-14,105,Weekend,April
4996,2024-04-14 02:00:00,0.493568,0.258212,0.677895,0.627889,0.535212,Normal,2024,4,14,Sunday,6,2024-04-14 02:00:00,2024-04-08,2024-04-14,105,Weekend,April
4997,2024-04-14 02:30:00,0.893818,0.431739,0.688926,0.508038,0.681099,Abnormal,2024,4,14,Sunday,6,2024-04-14 02:00:00,2024-04-08,2024-04-14,105,Weekend,April
4998,2024-04-14 03:00:00,0.509673,0.592927,0.366151,0.668218,0.710599,Normal,2024,4,14,Sunday,6,2024-04-14 03:00:00,2024-04-08,2024-04-14,105,Weekend,April


# Plot energy consumption 

1. Plot the electricity consumption at the following intervals: 
- half-hour 
- daily  
NB. When plotting in sns.lineplot set the estimator=None so it show the raw values without averaging. 
Eg. Day: each day has 48 readings. When plotting seaborn average the half-hourly electrcity consumption per day and the variability of the half-hour readings wihtin each day. But we want the raw data not the average.

1. Plot aggregated data at the following intervals: 
- daily
- weekly
- monthly  
NB. for aggregated data sum the electrcity consumption by day_of_year/week/month (daily_demand)

## Plot without aggregation

In [ ]:
# half-hour



In [ ]:
# daily

### Some insights:  
The profiles shows ...., 
with fluctutations .... 
BUT ....

## Aggregated Consumptions

In [ ]:
# daily consumption

#daily_demand=


In [ ]:
# weekly consumption

#weekly_demand 

In [ ]:
# monthly consumption

#monthly_demand 

### Some insights: 
Looking at the monthly energy consumption (510-600kWh/month) --> possibly is a ........  
[On average, the monthly consumption of a resindetial building - house with 4 people- is 290-425kWh/month] 

- Check the energy consumption the weekend vs weekday 

In [ ]:
# energy consumption per day name - sum
day_name_demand = df.groupby("Day_name")["Electricity_Consumed"].sum().reset_index()

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_name_demand['Day_name'] = pd.Categorical(day_name_demand['Day_name'], categories=day_order, ordered=True)
day_name_demand = day_name_demand.sort_values('Day_name')



In [ ]:
# energy consumption per day name - average
day_name_demand = df.groupby("Day_name")["Electricity_Consumed"].mean().reset_index()

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_name_demand['Day_name'] = pd.Categorical(day_name_demand['Day_name'], categories=day_order, ordered=True)
day_name_demand = day_name_demand.sort_values('Day_name')

